# International Faculty Development Program on _“Quantum Artificial Intelligence”_

#### Quantum Support Vector Machine (QSVM)

- Support Vector Machine (SVM) is a supervised machine learning algorithm used for both classification (and regression).
- The main objective of the SVM algorithm is to find the optimal hyperplane in an N-dimensional space that can separate the data points in different classes in the feature space.
- The hyperplane tries that the margin between the closest points of different classes should be as maximum as possible. This hyperplane is called the `the maximum-margin hyperplane/hard margin`.
- If number of features is $N$, the dimension of the hyperplane will be $N-1$.

![SVM](https://upload.wikimedia.org/wikipedia/commons/7/72/SVM_margin.png)

* Hyperplane: $\mathbf{w\cdot x} - b = 0$.
* Support vectors: Datapoints lies on $\mathbf{w\cdot x} - b = \pm1$.
* Margin: The distance between the support vector and hyperplane ($\frac{1}{||w||}$). The main objective of the support vector machine algorithm is to maximize the margin.  The wider margin indicates better classification performance.
* Classical SVM problem is an optimization problem. Using Lagrangian, one can convert it to a problem of solving a system of linear equation.

__Algorithm__

`Input:`
- Datapoints $\{\mathbf x_0,\mathbf x_1,\dots,\mathbf x_{n-1}\}$.
- Labels $\{y_0,y_1,\cdots,y_{n-1}\}$.

`Output:` $\mathbf{w}$ and $b$ such that $\mathbf{w}\cdot\mathbf{x}-b=0$ is the maximum-margin hyperplane .
1. Construct matrix $A=\begin{pmatrix}0&\mathbf 1^T\\\mathbf 1&K+\frac{\mathbb I}{\gamma}\end{pmatrix}$, where $K_{ij}=\mathbf x_i^T\cdot\mathbf x_j$ and user-specified $\gamma$ determines training-error.
2. Construct vector $\mathbf y = \begin{pmatrix}0\\y_0\\\vdots\\y_{n-1}\end{pmatrix}$.
3. Slove $A\mathbf z=\mathbf y$ to get $\mathbf z$. (Here we will use HHL algorithm.)
4. $b=z_0$, $\mathbf w=\sum_iz_i\mathbf x_{i-1}$ will give the required hyperplane.

In [ ]:
class HHL():
    
    '''class for HHL algorithm'''
    def __init__(self, A, b, n):
        '''__init__ method initiates a class.'''
        from math import log2
        
        self.matrix = A
        self.vector = b
        self.num_clock = n
        self.num_qubit = int(log2(len(b)))
        pass
    
    def create_circuit(self):
        '''Creates HHL circuit'''
        from qiskit import QuantumCircuit, ClassicalRegister, transpile
        from qiskit.circuit.library import PhaseEstimation as QPE
        from numpy import linalg
        
        # Initialize
        circ = QuantumCircuit(self.num_qubit)
        circ.initialize(self.vector/linalg.norm(self.vector))
        circ.barrier()
        # Perform QPE
        gate = self._phase_est(circ)
        circ.barrier()
        # Perform rotation
        self._rotation(circ)
        circ.barrier()
        # Uncompute QPE
        uncompute = QPE(self.num_clock, gate).to_gate()
        circ.append(uncompute.inverse(), range(self.num_clock+self.num_qubit)[::-1])
        circ.barrier()
        
        self.circuit = circ
        
        pass
    
    def solve(self):
        '''Solves the system of equatuin to get result'''
        from qiskit.quantum_info import partial_trace, DensityMatrix
        from numpy import matmul, linalg

        DM = DensityMatrix(self.circuit)
        value, DM = DM.measure([self.num_clock+self.num_qubit])
        if not value:
            self.solution = None
            return
        DM = partial_trace(DM, range(self.num_qubit, self.circuit.num_qubits))
        sv = DM.to_statevector(atol = .1).data
        sol = [[vec.real] for vec in sv]
        
        # Get x
        B = list(matmul(self.matrix, sol))
        i = 0
        while not self.vector[i]:
            i += 1
        norm_x = self.vector[i]/B[i]
        self.solution = [s*norm_x for s in sol]

    def _phase_est(self, qc):
        from qiskit import QuantumRegister
        from qiskit.circuit.library import HamiltonianGate, PhaseEstimation as QPE
        
        clock = QuantumRegister(self.num_clock, name = 'clock')
        qc.add_register(clock)

        # create gate U_A
        gate = HamiltonianGate(self.matrix, 1)

        # Perform QPE
        qc.append(QPE(self.num_clock, gate), range(self.num_clock+self.num_qubit)[::-1])

        return gate

    def _rotation(self, qc):
        from math import pi
        from qiskit import QuantumRegister
        
        ancilla = QuantumRegister(1, name = 'ancilla')
        qc.add_register(ancilla)

        # Perform rotation
        for i in range(self.num_qubit, self.num_clock+self.num_qubit):
            qc.cry(pi/2**i, i, ancilla)

In [ ]:
def generate_matrix(data_train, label_train, gamma = 0.7):
    from numpy import ones, eye
    
    m = len(data_train) + 1
    A = ones((m, m))
    A[0][0] = 0
    for i in range(1, m):
        for j in range(1, m):
            A[i][j] = dot(data_train[i-1], data_train[j-1])
            if i == j:
                A[i][j] += 1/gamma
    b = [0] + list(label_train)
    return A, b

Here we will randomly generate an artificial dataset. 

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

# Generate data
n_features = 2
train_size = 7
test_size = 50
centers = [(2.5, 0), (2.5, 2.5)]
data, target = make_blobs(n_samples = train_size+test_size, centers = centers, n_features = n_features, cluster_std = 0.5)

# Split data
data_train, data_test, label_train, label_test = train_test_split(data, target, train_size = train_size, test_size = test_size)

# Re-label from 0 to -1
for i in range(len(label_train)):
    if not label_train[i]:
        label_train[i] = -1

In [ ]:
# Visualize data
from matplotlib.pyplot import subplots, scatter, show, plot

_, ax = subplots(1, 1, figsize = (10, 5))

scatter(data[:,0], data[:,1], c = target)
show()

In [ ]:
from numpy import zeros, dot

# Find hyperplane
gamma = 1
A, b = generate_matrix(data_train, label_train, gamma)
hhl = HHL(A, b, 5)
hhl.create_circuit()
hhl.solve()
z = hhl.solution
w = zeros((1, len(data_train[0])))
for x, j in zip(data_train, z[1:]):
    w += list(j*x)
b = z[0]
print(f'Hyperplane to classify data is given by w.x=b with \nw = {w} and b = {b}')

In [ ]:
# Visualize the classification
_, ax = subplots(1, 1, figsize = (10, 5))

scatter(data[:,0], data[:,1], c = target)
plot(range(int(data[:,0].min()), int(data[:,0].max())+2), [(b-w[0][0]*x)/w[0][1] for x in range(int(data[:,0].min()), int(data[:,0].max())+2)])
ax.set_ylim(data[:,1].min(), data[:,1].max())
show()

In [ ]:
# Test SVM
acc = 0
for x, y in zip(data_test, label_test):
    if dot(w, x) + b < 0:
        if not y:
            acc += 1
    else:
        if y:
            acc += 1
print(acc/len(data_test))

If the datapoints are not linearly separable, they are mapped to higher dimension to make them linearly separable. The mathematical function, which is used to map the original input data points into high-dimensional feature spaces, is called Kernel.

![QKSVM](https://miro.medium.com/v2/resize:fit:4800/format:webp/0*b3zdsV3yPgRgV_sV)

Refer https://qiskit-community.github.io/qiskit-machine-learning/tutorials/03_quantum_kernel.html for Quantum Kernel Support Vector Machine (QKSVM).

#### Reference

1. Patrick Rebentrost, Masoud Mohseni, and Seth Lloyd, Phys. Rev. Lett. 113, 130503.
2. Aram W. Harrow, Avinatan Hassidim, and Seth Lloyd, Phys. Rev. Lett. 103, 150502.
3. https://qiskit-community.github.io/qiskit-machine-learning/tutorials/03_quantum_kernel.html